In [2]:
import pandas as pd
import numpy as np
from shapely import geometry

from shapely.ops import cascaded_union
import os

In [3]:
#To generate positions to image for tissue slices, select n points at the edge of the desired area for each slice
#and generate x*x tiles centered at the desired region. The tiles that are not overlapped with the desired region are removed. 
#points = pd.read_csv(r"D:\Data\AV\MOP_12092021\steve.txt", header = None, sep=',')

points1 = [[0,0],[500,500], [500,-500], [-500,-500],[-500,500]]
#points2 = [[-6799, 3946], [-6531, 4467], [-6264, 5138]]
#points3 = [[-11850, 7763], [-11541, 8429], [-11489, 8836], [-9450, 7763], [-9141, 8429], [-9089, 8836]]


points = np.array([points1])


fov_x_width = int(np.floor((123.0*0.75)/2))
fov_y_width = int(np.floor((200.0*0.75)/2))

In [4]:
temp = np.array(points1)
temp = np.transpose(temp)

In [5]:
print((np.min(temp[0]), np.max(temp[0])))
print((np.min(temp[1]), np.max(temp[1])))

(-500, 500)
(-500, 500)


In [6]:
points

array([[[   0,    0],
        [ 500,  500],
        [ 500, -500],
        [-500, -500],
        [-500,  500]]])

In [7]:
points.shape

(1, 5, 2)

In [8]:
grid = []
for x in range(-fov_x_width, fov_y_width+1):
  for y in range(-fov_x_width, fov_y_width+1):
    if x % 2 == 0:
        grid.append((x*2*fov_x_width, y*2*fov_y_width))
    else:
        grid.append((x*2*fov_x_width, -y*2*fov_y_width))
    
grid_R =  grid[::-1]
grid = np.array(grid)
grid_R = np.array(grid_R)

In [9]:
grid_R

array([[  6900, -11250],
       [  6900, -11100],
       [  6900, -10950],
       ...,
       [ -4232,  -6600],
       [ -4232,  -6750],
       [ -4232,  -6900]])

In [10]:
n = points.shape[0] #number of positions you selected for each slice to define the region of interest
positions = np.empty((0,2))
for i in range(0,n):
    tissue = geometry.Polygon(points[i])
    center = np.mean(points[i], axis = 0)
    grids = grid_R + center
    saved = np.empty((0,2))
    for j in range(0,len(grids)):
        area = [[grids[j][0]-fov_x_width,grids[j][1]-fov_y_width],[grids[j][0]-fov_x_width,grids[j][1]+fov_y_width],[grids[j][0]+fov_x_width,grids[j][1]+fov_y_width],[grids[j][0]+fov_x_width,grids[j][1]-fov_y_width]]
        tile = geometry.Polygon([[p[0], p[1]] for p in area])
        if tissue.intersects(tile) == True:
            saved = np.concatenate((saved,np.array([grids[j]])), axis = 0)
    positions = np.concatenate((positions,saved))

In [11]:
len(positions)

66

In [12]:
positions

array([[ 460., -450.],
       [ 460., -300.],
       [ 460., -150.],
       [ 460.,    0.],
       [ 460.,  150.],
       [ 460.,  300.],
       [ 460.,  450.],
       [ 368.,  450.],
       [ 368.,  300.],
       [ 368.,  150.],
       [ 368.,    0.],
       [ 368., -150.],
       [ 368., -300.],
       [ 368., -450.],
       [ 276., -450.],
       [ 276., -300.],
       [ 276., -150.],
       [ 276.,    0.],
       [ 276.,  150.],
       [ 276.,  300.],
       [ 184.,  300.],
       [ 184.,  150.],
       [ 184.,    0.],
       [ 184., -150.],
       [ 184., -300.],
       [ 184., -450.],
       [  92., -450.],
       [  92., -300.],
       [  92., -150.],
       [  92.,    0.],
       [  92.,  150.],
       [   0.,    0.],
       [   0., -150.],
       [   0., -300.],
       [   0., -450.],
       [ -92., -450.],
       [ -92., -300.],
       [ -92., -150.],
       [ -92.,    0.],
       [ -92.,  150.],
       [-184.,  300.],
       [-184.,  150.],
       [-184.,    0.],
       [-18

In [13]:
np.savetxt(r"C:\Users\Zhuang\storm_control\storm_control\dave\xml_generators\Aditya\positions.txt",positions, delimiter=',')